# Dask - GeoPandas Example

*Rob Knapen, Wageningen Environmental Research*
<br>

A notebook for trying out the Dask framework (as alternative to PySpark) with GeoPandas. This could be useful for processing large datasets of species observations.

### Import Python packages

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'

import pandas as pd
import numpy as np

import dask.dataframe as dd
import dask.array as da
import dask.bag as db
from dask.distributed import Client

import geopandas as gp
import dask_geopandas as dgp

### Start a Dask client
Get a client for the dummy local Dask 'cluster', and the IP for the dashboard.

In [ ]:
dask_client = Client()
dask_client

### Load sample species observation data
As an example a dataset from the Dutch 'Nationale Databank Flora en Fauna' (ndff.nl). While we are hoping for this to be available as open data (soon), it is not yet. However, we have permission to use it for the FAIRiCUBE EU project.

In [ ]:
# the NDFF datafile to process
species_filename = "../../../local/data/ndff/broedvogels_2016.csv"

# the columns to drop right away
always_drop_cols_from_source = None

In [ ]:
# load the data into a regular pandas dataframe
species_df = pd.read_csv(species_filename, header='infer', sep=';', on_bad_lines='warn')

# remove not needed columns
if always_drop_cols_from_source:
    species_df.drop(columns=always_drop_cols_from_source, inplace=True)

# remove the crs prefix from the wkt data
species_df['wkt_excl_crs'] = species_df['wkt'].map(lambda x: x.split(';')[1], na_action='ignore')
species_df.drop(columns=['wkt'], inplace=True)
species_df.rename(columns={'wkt_excl_crs': 'wkt'}, inplace=True)

species_df.head(5)

In [ ]:
# load the data into a dask dataframe

# read a Dask DataFrame
species_dd = dd.read_csv(
    species_filename,
    header='infer',
    sep=';',
    on_bad_lines='warn',
    dtype={ 'orig_abundance': 'object'} # because of '*' used as abundance value
)

# remove not needed columns, note that Dask DataFrames are immutable (unlike regular Pandas)
if always_drop_cols_from_source:
    species_dd = species_dd.drop(columns=always_drop_cols_from_source)

# remove the crs prefix from the wkt data
# note that map with a custom function needs additional meta info
species_dd['wkt_excl_crs'] = species_dd['wkt'].map(
    lambda x: x.split(';')[1],
    na_action='ignore',
    meta=pd.Series(dtype='str'))

species_dd = species_dd.drop(columns=['wkt'])
species_dd = species_dd.rename(columns={'wkt_excl_crs': 'wkt'})

species_dd.head(5)

In [ ]:
# Dask is lazy, need to call compute to get the result from a task graph
graph = species_dd['sci_name'].value_counts(sort=True, dropna=True)
graph.compute()

### Create a GeoPandas DataFrame
The observations have spatial attributes, so lift them into a GeoPandas DataFrame to be able to process them.

Note that there is a dask-geopandas package that bridges Dask with GeoPandas.

In [ ]:
# construct a GeoDataFrame, with the data using the Dutch RD coordinate reference system

# note that we used the pandas dataframe
gs = gp.GeoSeries.from_wkt(species_df['wkt'])
species_gdf = gp.GeoDataFrame(species_df, geometry=gs, crs='EPSG:28992')

# transform the dataset to the more common WGS84 (unprojected) CRS
species_gdf.to_crs(crs="EPSG:4326", inplace=True)
species_gdf.drop(columns=['wkt'], inplace=True)

species_gdf.head(5)

In [ ]:
# spatially select observations within an area of interest
aoi_gdf = species_gdf.cx[4.0:4.1, 51.0:51.5]
aoi_gdf

In [ ]:
# note that displaying large datasets in very time-consuming
aoi_gdf.explore()

### Create a Dask GeoDataFrame
Turn a regular geodataframe into a Dask geodataframe that support lazy graphs computed on a cluster.

In [ ]:
%%capture --no-display
# (hides warning about sending large graph)

# create a dask geodataframe
species_gdd = dgp.from_geopandas(species_gdf, npartitions=4)
species_gdd.compute()

In [ ]:
%%capture --no-display
# (hides warning about sending large graph)

species_gdd['sci_name'].value_counts(sort=True, dropna=True).compute()

In [ ]:
%%capture --no-display
# (hides warning about area calculation on non-projected data)

species_gdd.geometry.area.compute()

In [ ]:
dask_client.close()